# NEMPY Constraints Analysis

Jupyter Notebook <br>Version: 1.1 <br> Updated: 22/03/2022 <br>
Author: Declan Heim <br>
Contact: [Email](mailto:d.heim@unsw.edu.au)<br>
GitHub Ref: [nempy_constraints_v1](https://github.com/dec-heim/nempy_constraints_v1)

---

## Introduction & Background

This notebook serves as an example of how constraints can be analysed using the open-source python simulator of AEMO's dispatch process, 'nempy'. This tool was developed by Nick Gorman (UNSW CEEM) and is expanded upon using a [modified version](https://github.com/dec-heim/nempy_constraints_v1) with additional functionality to extract and manipulate constraints. These changes are not yet available via the official nempy release through pip, however updates to integrate and streamline these features into the official release of nempy are planned.

The purpose of analysing constraints in this regard is to convey a deeper understanding and discussion around the impact of constraints in NEM dispatch. This example demonstrates the outcomes that could be seen in the market by alleviating a specific constraint, both with respect to any changes in market prices and the volume (MW) dispatched for each unit involved in the constraint. This type of analysis is possible by re-simulating the same dispatch interval and modifying the inputs that define the historically binding constraint. Such conterfactual work requires the use of a dispatch simulator, unlike higher-level observations which may demonstrate the fact that units have been constrained but are unable to determine what otherwise may have happened.

The example provided assumes the reader has some foundational understanding of the NEM dispatch process, how constraints are defined, the types of constraints and so forth. Hence it can be seen as an intermediate example. Explainations of historical dispatch procedures in nempy are omitted given there is abundant information [covered in this example](https://nempy.readthedocs.io/en/latest/examples.html#detailed-recreation-of-historical-dispatch). To gain a thorough understanding of the work presented here, one should first understand simplier examples of NEM dispatch and have some insight as to how nempy can be used, as provided in [Nempy Documentation.](https://nempy.readthedocs.io/) Some helpful resources to assist in understanding constraints in the NEM are linked below.

### Acknowledgement:
Special thanks to Nick Gorman at UNSW for his support towards this analysis piece and development of nempy more broadly.

---

### Useful References:
<font color=blue>**Relevant nempy material:**</font>
- [UNSW-CEEM Nempy Github](https://github.com/UNSW-CEEM/nempy)
- [Nempy Documentation](https://nempy.readthedocs.io/)

<font color=blue>**Relevant nempy examples:**</font>
- [Nempy: Simple Examples 1-5](https://nempy.readthedocs.io/en/latest/examples.html#)
- [Nempy: Detailed Recreation of Historical Dispatch](https://nempy.readthedocs.io/en/latest/examples.html#detailed-recreation-of-historical-dispatch)

<font color=blue>**Relevant material for understanding constraints:**</font>
- [AEMO Constraint Implementation Guidelines (2015)](https://www.aemo.com.au/Electricity/National-Electricity-Market-NEM/Security-and-reliability/-/media/943DDD419E5942B8993CCA8EA201C37E.ashx)
- [AEMO Constraint Naming Guidelines (2013)](https://www.aemo.com.au/-/media/Files/Electricity/NEM/Security_and_Reliability/Congestion-Information/2016/Constraint-Naming-Guidelines.pdf)
- [Example of Marginal Value Calculations of 'X5' constraint, by Allan O'Neil (WattClarity)](https://wattclarity.com.au/articles/2020/12/casestudy-followup-x5-constraint/)

<font color=blue>**Specific material for the constraint that this example draws upon:**</font>
- [Example Analysis of Constraint Impact on PV, by Jack Simpson](https://jacksimpson.co/exploring-the-impact-of-constraints-on-a-solar-farm-in-the-national-electricity-market/)
- [AEMO Feb 2021 Monthly Constraint Report](https://www.aemo.com.au/-/media/files/electricity/nem/security_and_reliability/congestion-information/statistics/2021/monthly-constraint-report-february-2021.pdf?la=en).
---

## Example 1 - Investigating the impact of constrained line 94T - Feb 2021
### Event Description
This example highlights historical events of the constrained line 94T representing Molong to Orange North in NSW during February 2021. Specifically this event was noted earlier by [J. Simpson.](https://jacksimpson.co/exploring-the-impact-of-constraints-on-a-solar-farm-in-the-national-electricity-market/) and can further be found through AEMO's [monthly reporting process.](https://www.aemo.com.au/-/media/files/electricity/nem/security_and_reliability/congestion-information/statistics/2021/monthly-constraint-report-february-2021.pdf?la=en)


Specific details of which units are involved in the constraint is provided in the latter sections of this notebook, demonstrating how such information can be extracted from nempy. In the meantime, we can contextualise this constraint as having a signficant impact on a number of solar farms located in Central-West NSW as flagged by the earlier references.

---

### Notebook Format
The subsequent sections of the notebook are formatted under section and sub-section headings, each a distinct step in this analysis piece. A brief overview is that this notebook entails:

- [Section A: Building the model in nempy](#A)
- [Section B: Historical results with the constraint imposed](#B)
- [Section C: Counter-factual results after removing the constraint](#C)


<a id="A"></a>
## Section A. Model Building
### A1. Import Python Packages


In [5]:
import pandas as pd
!pip install db-sqlite3
import sqlite3

# The forked nempy market dispatch engine is loaded
!pip install --user --upgrade git+https://github.com/dec-heim/nempy_constraints_v1.git --no-warn-script-location
from nempy import markets, time_sequential
from nempy.historical_inputs import loaders, mms_db, \
    xml_cache, units, demand, interconnectors, constraints

# Plotting packages
!pip install plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

n.b. Printout of installation will appear here but removed here


---

### A2. Retrieve + Prepare Historical Data from AEMO

The data for this example has already been extracted and prepared for nempy locally. The data was found for the **15th February 2021** and includes a range of parameters, to name a few:
- unit details
- unit availability
- volume bids
- price bids
- unit ramp rate constraints
- interconnector models with loss equations
- regional demand constraints
- various other constraints such as FCAS, generic constraint sets

Further steps on how to download the data from AEMO MMS and configure this for nempy are provided in the [Detailed Recreation of Historical Dispatch Example](https://nempy.readthedocs.io/en/latest/examples.html#detailed-recreation-of-historical-dispatch) within nempy documentation.

In [6]:
con = sqlite3.connect('feb_2021_mms.db')
mms_db_manager = mms_db.DBManager(connection=con)
xml_cache_manager = xml_cache.XMLCacheManager('feb_2021_cache')

raw_inputs_loader = loaders.RawInputsLoader(
    nemde_xml_cache_manager=xml_cache_manager,
    market_management_system_database=mms_db_manager)

---

### A3. Define Dispatch Intervals
For this specific example, we will look at one specific dispatch interval for **15th February 2021** which is the interval at **11:00:00** (time-ending). This is formatted as a list for ease of later assessing multiple intervals.

Once the interval has been defined, the subsequent functions are called to load the respective data from our previously downloaded database.

In [7]:
dispatch_intervals = ['2021/02/15 11:00:00']
                      
interval = dispatch_intervals[0]

In [8]:
outputs = []
raw_inputs_loader.set_interval(interval)
unit_inputs = units.UnitData(raw_inputs_loader)
interconnector_inputs = interconnectors.InterconnectorData(raw_inputs_loader)
constraint_inputs = constraints.ConstraintData(raw_inputs_loader)
demand_inputs = demand.DemandData(raw_inputs_loader)


---
### A4. Define Market Data
The subsequent data loaded into the nempy market object follows the [Detailed Recreation of Historical Dispatch](https://nempy.readthedocs.io/en/latest/examples.html#detailed-recreation-of-historical-dispatch) so we will omit this detail here for brevity and highlight only the generic constraints which are of interest in this analysis.

In [9]:
# Define market, unit info
unit_info = unit_inputs.get_unit_info()
market = markets.SpotMarket(market_regions=['QLD1', 'NSW1', 'VIC1',
                                            'SA1', 'TAS1'],
                            unit_info=unit_info)

# Set volume, price bids
volume_bids, price_bids = unit_inputs.get_processed_bids()
market.set_unit_volume_bids(volume_bids)
market.set_unit_price_bids(price_bids)


# Set bid in capacity limits
unit_bid_limit = unit_inputs.get_unit_bid_availability()
market.set_unit_bid_capacity_constraints(unit_bid_limit)
cost = constraint_inputs.get_constraint_violation_prices()['unit_capacity']
market.make_constraints_elastic('unit_bid_capacity', violation_cost=cost)

# Set limits provided by the unconstrained intermittent generation
# forecasts. Primarily for wind and solar.
unit_uigf_limit = unit_inputs.get_unit_uigf_limits()
market.set_unconstrained_intermitent_generation_forecast_constraint(
    unit_uigf_limit)
cost = constraint_inputs.get_constraint_violation_prices()['uigf']
market.make_constraints_elastic('uigf_capacity', violation_cost=cost)

# Set unit ramp rates.
ramp_rates = unit_inputs.get_ramp_rates_used_for_energy_dispatch()
market.set_unit_ramp_up_constraints(
    ramp_rates.loc[:, ['unit', 'initial_output', 'ramp_up_rate']])
market.set_unit_ramp_down_constraints(
    ramp_rates.loc[:, ['unit', 'initial_output', 'ramp_down_rate']])
cost = constraint_inputs.get_constraint_violation_prices()['ramp_rate']
market.make_constraints_elastic('ramp_up', violation_cost=cost)
market.make_constraints_elastic('ramp_down', violation_cost=cost)

# Set unit FCAS trapezium constraints.
unit_inputs.add_fcas_trapezium_constraints()
cost = constraint_inputs.get_constraint_violation_prices()['fcas_max_avail']
fcas_availability = unit_inputs.get_fcas_max_availability()
market.set_fcas_max_availability(fcas_availability)
market.make_constraints_elastic('fcas_max_availability', cost)
cost = constraint_inputs.get_constraint_violation_prices()['fcas_profile']
regulation_trapeziums = unit_inputs.get_fcas_regulation_trapeziums()
market.set_energy_and_regulation_capacity_constraints(regulation_trapeziums)
market.make_constraints_elastic('energy_and_regulation_capacity', cost)
scada_ramp_down_rates = unit_inputs.get_scada_ramp_down_rates_of_lower_reg_units()
market.set_joint_ramping_constraints_lower_reg(scada_ramp_down_rates)
market.make_constraints_elastic('joint_ramping_lower_reg', cost)
scada_ramp_up_rates = unit_inputs.get_scada_ramp_up_rates_of_raise_reg_units()
market.set_joint_ramping_constraints_raise_reg(scada_ramp_up_rates)
market.make_constraints_elastic('joint_ramping_raise_reg', cost)
contingency_trapeziums = unit_inputs.get_contingency_services()
market.set_joint_capacity_constraints(contingency_trapeziums)
market.make_constraints_elastic('joint_capacity', cost)

# Set interconnector definitions, limits and loss models.
interconnectors_definitions = \
    interconnector_inputs.get_interconnector_definitions()
loss_functions, interpolation_break_points = \
    interconnector_inputs.get_interconnector_loss_model()
market.set_interconnectors(interconnectors_definitions)
market.set_interconnector_losses(loss_functions,
                                  interpolation_break_points)

# Add FCAS market constraints.
fcas_requirements = constraint_inputs.get_fcas_requirements()
market.set_fcas_requirements_constraints(fcas_requirements)
violation_costs = constraint_inputs.get_violation_costs()
market.make_constraints_elastic('fcas', violation_cost=violation_costs) 


---

### A5. Relevant Constraints Data
A few key dataframes should be highlighted which are directly relevant to the constraint equation we will assess.

<ol>
    <li>The <b>Generic RHS dataframe:</b> This data contains columns specifying the constraint set (as referred to in nempy yet more 'correctly' referred to as constraint equation), the RHS value, and the type. In simple terms, the constraint equation defines the name of the constraint, the RHS value is the limit (generally maximum, minimum, or equal to) value which the sum of the LHS arguments must comply with, and the type is the defined mathematically operator (less than and equal, more than and equal, or equal to). </li>
    <li>The <b>Unit Generic LHS dataframe:</b> This dataframe maps each unit assosciated with each constraint equation. The columns specified include the constraint 'set', the unit that is found in the LHS of the constraint equation, the service involved (e.g. energy), and the LHS coefficient assosciated with the unit (effectively the multiplier for the unit dispatch volume in MWs).
    <li>The <b>Interconnector Generic LHS dataframe:</b> This dataframe is akin to the unit generic lhs above however maps the interconnectors found as LHS arguments for specific constraints and their coefficients.    
</ol>

A final note is that violation costs apply to each constraint set here. These costs have been specified in the prior code section with the line <code>violation_costs = constraint_inputs.get_violation_costs()</code> . In essence these generic constraints are 'soft' contraints that could theoretically be violated but the dispatch engine would assosciate (usually an extremely high cost) for each MW the constraint equation is violated. Such a structure allows the dispatch engine to violate constraints in a preferred order, e.g. FCAS requirements violated before thermal constraints. 

In [10]:
# Add generic constraints, RHS parameters
generic_rhs = constraint_inputs.get_rhs_and_type_excluding_regional_fcas_constraints()
market.set_generic_constraints(generic_rhs)
market.make_constraints_elastic('generic', violation_cost=violation_costs)

print(f"\nThe Generic Constraints RHS are defined for {interval} as:")
generic_rhs


The Generic Constraints RHS are defined for 2021/02/15 11:00:00 as:


,set,rhs,type
0,#BANGOWF1_E,11.000000,<=
1,#BBTHREE3_E,25.000000,<=
2,#BULGANA1_E,100.000000,<=
3,#COHUNSF1_E,0.000000,<=
4,#COLWF01_E,146.000000,<=
...,...,...,...
822,V_OWF_TGTSNRBHTN_30,10030.000000,<=
823,V_S_HEYWOOD_UFLS,142.735569,<=
824,V_S_NIL_ROCOF,408.000000,<=
825,V_T_NIL_BL1,478.000000,<=


In [11]:
# Add generic constraints, LHS coefficients DUIDs
unit_generic_lhs = constraint_inputs.get_unit_lhs()
market.link_units_to_generic_constraints(unit_generic_lhs)

print(f"\nThe Generic Constraints DUID LHS coefficients are defined for {interval} as:")
unit_generic_lhs


The Generic Constraints DUID LHS coefficients are defined for 2021/02/15 11:00:00 as:


,set,unit,service,coefficient
0,#BANGOWF1_E,BANGOWF1,energy,1.0
1,#BBTHREE3_E,BBTHREE3,energy,1.0
2,#BULGANA1_E,BULGANA1,energy,1.0
3,#COHUNSF1_E,COHUNSF1,energy,1.0
4,#COLWF01_E,COLWF01,energy,1.0
...,...,...,...,...
9378,V_MTGBRAND_44WT,MTGELWF1,energy,1.0
9379,V_MWWF_GFT1_5,MUWAWF1,energy,1.0
9380,V_OAKHILL_TFB_42,OAKLAND1,energy,1.0
9381,V_OWF_NRB_0,OAKLAND1,energy,1.0


In [12]:
# Add generic constraints, LHS coeffients ICs
interconnector_generic_lhs = constraint_inputs.get_interconnector_lhs()
market.link_interconnectors_to_generic_constraints(
    interconnector_generic_lhs)

print(f"\nThe Generic Constraints Interconnector LHS coefficients are defined for {interval} as:")
interconnector_generic_lhs


The Generic Constraints Interconnector LHS coefficients are defined for 2021/02/15 11:00:00 as:


,set,interconnector,coefficient
0,DATASNAP,N-Q-MNSP1,1.0
1,DATASNAP_DFS_LS,N-Q-MNSP1,1.0
2,DATASNAP_DFS_NCAN,N-Q-MNSP1,1.0
3,DATASNAP_DFS_NCWEST,N-Q-MNSP1,1.0
4,DATASNAP_DFS_NNTH,N-Q-MNSP1,1.0
...,...,...,...
673,V^^S_PAVC_MAXG-DS,V-SA,1.0
674,V_S_HEYWOOD_UFLS,V-SA,1.0
675,V_S_NIL_ROCOF,V-SA,1.0
676,V_T_NIL_BL1,T-V-MNSP1,-1.0


---

### A6. Set Demand Requirements and Dispatch the Market
The final stage of this model building section is to define the regional demand requirements and run the market dispatch. 

In [13]:
# Set the operational demand to be met by dispatch.
regional_demand = demand_inputs.get_operational_demand()
market.set_demand_constraints(regional_demand)

print(f"\nThe Demand to be met for {interval} is:")
regional_demand


The Demand to be met for 2021/02/15 11:00:00 is:


,region,demand
0,NSW1,6986.08
1,QLD1,6249.63
2,SA1,730.55
3,TAS1,1048.68
4,VIC1,3749.57


In [14]:
market.dispatch()

---
<a id="B"></a>
## Section B - Initial Results (considered all historical constraints)
<b><font color=blue>This section applies additional functionality of the [modified nempy package](https://github.com/dec-heim/nempy_constraints_v1)</font></b>
### B1. Market Price Outcomes
The cleared market prices per region for the simulated historical dispatch interval are found below.

In [15]:
prices = market.get_energy_prices()

print(f"\nThe Cleared Market Prices per region for {interval} are:")
prices


The Cleared Market Prices per region for 2021/02/15 11:00:00 are:


,region,price
0,NSW1,34.616510
1,QLD1,32.670000
2,SA1,4.646826
3,TAS1,32.684000
4,VIC1,5.010000


---

### B2. Identifying the Binding Constraints
The binding constraints from the dispatch interval can be found by calling the function <code>market.get_constraint_marginal_values</code>.
This function searches our previously defined constraint inputs which we defined in the dataframes called <b> Generic RHS</b>, <b>Unit Generic LHS</b>, and <b>Unit Interconnector LHS</b>. For each generic constraint that has been applied to the dispatch process, the marginal value ("MV") is retrieved from the python mixed-integer-linear-program (mip) as a 'byproduct' in a sense of optimising the most economical dispatch.

In simple terms, the MV can be thought of as the amount by which the system dispatch cost would change should the constraint be relaxed by 1 MW. An example might be allowing more powerflow through a network line linking two nodes. Generally, we have a LHS <= RHS structure for thermal constraints. By increasing the RHS by 1MW, we would allow 1MW more power to be transferred through the line. Let's say this is 1MW more power from a cheap solar farm, so we no longer have to run a more expensive gas generator which is located somewhere else in the network. As such, the MV in this case would be negative, signifying that there is a cost reduction on the system dispatch. <b> This is not to be confused with the market clearing price </b>... more on that later. A final note is that if a constraint is not binding, it would have a zero MV. The unit outputs or interconnector flows on the LHS are not being restricted, so it is indifferent to relaxing the RHS by 1MW. 

The concept of marginal values is captured well in detail in the [Example of Marginal Value Calculations of 'X5' constraint, by Allan O'Neil (WattClarity)](https://wattclarity.com.au/articles/2020/12/casestudy-followup-x5-constraint/)


In [16]:
marginal_values = market.get_constraint_marginal_values()
marginal_values

,set,constraint_id,type,rhs,slack,marginal_value
24,#MWPS2PV1_E,2420,<=,0.000000,-0.000000e+00,-1004.646826
25,#MWPS4PV1_E,2421,<=,0.000000,-0.000000e+00,-1004.646826
27,#PPCCGT_D_E,2423,=,175.000000,0.000000e+00,14995.353174
29,#TORRB1_D_E,2425,=,40.000000,7.105427e-15,14995.353174
31,#WARWSF1_E,2427,<=,16.000000,0.000000e+00,-1032.670000
36,$CALL_B_1,2432,=,350.000000,0.000000e+00,14967.330000
109,F_T_AUFLS2_R6,2505,<=,0.001000,0.000000e+00,-8.050000
177,N>>N-NIL_94T_947,2573,<=,131.291525,0.000000e+00,-492.298968
213,N>>N-PKWL_94K_2,2609,<=,143.032866,0.000000e+00,-1034.616510
252,N^^N_NIL_2,2648,<=,450.535552,0.000000e+00,-1080.479531


The results above produce a dataframe identifying the constraint equations with non-zero marginal values. The columns here include the 'set', constraint id, type, rhs, slack and marginal_value. The only fields of concern to us in this example are the constraint 'set' or name and the fact that it has a non-zero MV indicating it is binding.

In this specific dispatch interval we notice a total of 14 binding constraints. To identify what each of these constraints mean, refer to the [AEMO Constraint Naming Guidelines (2013)](https://www.aemo.com.au/-/media/Files/Electricity/NEM/Security_and_Reliability/Congestion-Information/2016/Constraint-Naming-Guidelines.pdf).

---
### B3. Retrieve information about the 'N>>N-NIL_94T_947' constraint
We will select one binding constraint from this dispatch interval to analyse. In this example the <b>N>>N-NIL_94T_947</b> constraint. From which we can compare the earlier analysis of this occurance in the [Example by J. Simpson](https://jacksimpson.co/exploring-the-impact-of-constraints-on-a-solar-farm-in-the-national-electricity-market/)

Deciphering the naming guidelines this can be interpreted as:
- N: New South Wales
- <string>>>: Thermal overload of a network element</string>
- N-NIL: System Normal
- 94T: The network element being line 94T (Molong to Orange North)
- 947: The 'tripping' element which would cause the concerned line to overload, here being 947 (Wellington to Orange North).
Extracting only the information relevant to this constraint can be done slicing the dataframe as below.

Or concisely described by AEMO as: <b>"Out= Nil, avoid O/L Molong to Orange North (94T) on trip of Wellington to Orange North (947), Feedback"</b> A good time to also mention AEMO's reporting on this specific event occuring during Feb 2021 in it's [Monthly Constraint Report](https://www.aemo.com.au/-/media/files/electricity/nem/security_and_reliability/congestion-information/statistics/2021/monthly-constraint-report-february-2021.pdf?la=en).

Subsequently, the <code>market.get_constraint_mapping()</code> function is useful in collecting all the LHS terms involved in this constraint. Currently this returns a dict format of one dataframe of unit lhs, and one dataframe of interconnector lhs. Given it is a thermal constraint of line 94T, no interconnectors are found in the LHS so the latter return is an empty dataframe. We therefore are interested in the units lhs involved.

In [17]:
constraint_rhs_info = marginal_values[marginal_values['set'] == 'N>>N-NIL_94T_947'].reset_index(drop=True)
constraint_rhs_info

,set,constraint_id,type,rhs,slack,marginal_value
0,N>>N-NIL_94T_947,2573,<=,131.291525,0.0,-492.298968


In [18]:
constraintmap = market.get_constraint_mapping('N>>N-NIL_94T_947')
constraintmap['units']

,set,unit,service,coefficient
2548,N>>N-NIL_94T_947,BERYLSF1,energy,0.1355
2549,N>>N-NIL_94T_947,BODWF1,energy,0.2025
2550,N>>N-NIL_94T_947,GOONSF1,energy,0.5049
2551,N>>N-NIL_94T_947,JEMALNG1,energy,0.4273
2552,N>>N-NIL_94T_947,MANSLR1,energy,0.8803
2553,N>>N-NIL_94T_947,NEVERSF1,energy,0.2379
2554,N>>N-NIL_94T_947,NYNGAN1,energy,0.2379
2555,N>>N-NIL_94T_947,PARSF1,energy,0.5049


Note there are 8 unit DUIDs which are LHS terms involved in this binding constraint for this specific interval. Each of these can be identified as:
- Beryl Solar Farm (BERYLSF1) 
- Bodangora Wind Farm (BODWF1)
- Goonumbla Solar Farm (GOONSF1)
- Jemalong Solar Project (JEMALNG1)
- Manildra Solar Farm (MANSLR1)
- Nevertire Solar Farm (NEVERSF1)
- Nyngan Solar Plant (NYNGAN1)
- Parkes Solar Farm (PARSF1)

We can in fact verify that the constraint is binding by summating each units dispatched volume that contributes to the total LHS arguement. Firstly, let's extract the dispatch values for these units and save this matrix as <code>units_lhs_vals</code>.

In [19]:
units = market.get_unit_dispatch()
units_lhs_vals = units[units['unit'].isin(constraintmap['units']['unit'])]
units_lhs_vals

,unit,service,dispatch
46,BERYLSF1,energy,64.437253
52,BODWF1,energy,42.525000
185,GOONSF1,energy,69.000000
265,JEMALNG1,energy,14.998500
377,MANSLR1,energy,0.000000
428,NEVERSF1,energy,98.440000
431,NYNGAN1,energy,101.043000
438,PARSF1,energy,50.000000


Now we can merge this dataframe containing the dispatch volume (MW) and the earlier dataframe of the lhs coefficient factors.

By multiplying each unit's LHS dispatch value by the LHS coefficient, which we are denoting here as LHS contribution or 'contr', we arrive at the total summation of LHS arguments.

In [20]:
line94t_mtx = pd.merge(left=units_lhs_vals.loc[:,['unit','dispatch']],\
                       right=constraintmap['units'].loc[:,['unit','coefficient']],on='unit')

line94t_mtx['lhs_contr'] = line94t_mtx['dispatch']*line94t_mtx['coefficient']

line94t_mtx

,unit,dispatch,coefficient,lhs_contr
0,BERYLSF1,64.437253,0.1355,8.731248
1,BODWF1,42.525000,0.2025,8.611313
2,GOONSF1,69.000000,0.5049,34.838100
3,JEMALNG1,14.998500,0.4273,6.408859
4,MANSLR1,0.000000,0.8803,0.000000
5,NEVERSF1,98.440000,0.2379,23.418876
6,NYNGAN1,101.043000,0.2379,24.038130
7,PARSF1,50.000000,0.5049,25.245000


In [21]:
print(f"\n For the constraint at time: {interval}")
print(f"\n The sum of the LHS is {line94t_mtx['lhs_contr'].sum():.4f}")
print(f"\n The RHS limit is {constraint_rhs_info['rhs'][0]:.4f}")


 For the constraint at time: 2021/02/15 11:00:00

 The sum of the LHS is 131.2915

 The RHS limit is 131.2915


Hence, we have checked these values to be equal as suggested by the non-zero marginal value which indicates the constraint is binding.

---
<a id="C"></a>
## Section C - Counterfactual Analysis (removing the constraint)
<b><font color=blue>This section applies additional functionality of the [modified nempy package](https://github.com/dec-heim/nempy_constraints_v1)</font></b>
### C1. Removing the 'N>>N-NIL_94T_947' constraint
Having identified and found this constraint to be binding in our historical simulation of dispatch, this section now explains how the constraint can be removed and reveals the impact that this has on market outcomes. Until now, we could only gauge the impact of the constraint on the 'total system optimisation' which was indicated by the marginal value. Now we will gain a sense of market prices, changes in unit dispatch and so forth.

The function <code>market.remove_generic_cosntraint_set()</code> allows the user to specify a generic constraint to remove from the dispatch process. This function is called after all other constraints have been specified in the ealier Model Building process shown in Section A. In other words, a final function call before dispatching the market.

**In this section we will denote each python variable saved with a "_r" to signfiy the removal of the constraint.**

In [22]:
market.remove_generic_constraint_set('N>>N-NIL_94T_947')
market.dispatch()

There are two ways we can identify that the constraint has been successfully removed. Firstly, if we again call the <code>market.get_constraint_marginal_values()</code> function we notice the 'N>>N-NIL_94T_947' no longer appears. Given this is the only change enacted since running the historical dispatch the first time, we can be satified that is no longer exists otherwise it would have binded and have a non-zero MV shown here.

A second more convincing check would be to call the generic constraint variables stored in the market object which are processed by the mip solver. The return here shows no entry of RHS arguments for 'N>>N-NIL_94T_947'. Similarly, you could call the LHS objects mentioned in Section A5.

In [23]:
marginal_values_r = market.get_constraint_marginal_values()
marginal_values_r

,set,constraint_id,type,rhs,slack,marginal_value
25,#MWPS4PV1_E,2421,<=,0.000000,-0.000000e+00,-1004.646826
27,#PPCCGT_D_E,2423,=,175.000000,0.000000e+00,14995.353174
29,#TORRB1_D_E,2425,=,40.000000,7.105427e-15,14995.353174
35,$BLOWERNG,2431,=,33.000000,0.000000e+00,-1034.447395
36,$CALL_B_1,2432,=,350.000000,0.000000e+00,14967.489606
109,F_T_AUFLS2_R6,2505,<=,0.001000,0.000000e+00,-8.050000
213,N>>N-PKWL_94K_2,2609,<=,143.032866,0.000000e+00,-1034.447395
252,N^^N_NIL_2,2648,<=,450.535552,0.000000e+00,-1080.302919
253,N^^N_NIL_3,2649,<=,346.585082,0.000000e+00,-292.819441
300,Q>NIL_YLMR,2696,<=,100.376067,0.000000e+00,-85.220394


In [24]:
solver_inputs = market._constraints_rhs_and_type['generic']
solver_inputs[solver_inputs['set'] == 'N>>N-NIL_94T_947']

,set,constraint_id,type,rhs,slack,marginal_value


---
### C2. Assessing the impact of the constraint removal - Dispatch Outcomes

As per the previous Section B, we can collect the unit dispatch outcomes for those units involved in the LHS of the constraint.

In [25]:
units_r = market.get_unit_dispatch()
units_lhs_vals_r = units_r[units_r['unit'].isin(constraintmap['units']['unit'])]
units_lhs_vals_r

,unit,service,dispatch
46,BERYLSF1,energy,81.1280
52,BODWF1,energy,42.5250
185,GOONSF1,energy,69.0000
265,JEMALNG1,energy,14.9985
377,MANSLR1,energy,0.0000
428,NEVERSF1,energy,98.4400
431,NYNGAN1,energy,101.0430
438,PARSF1,energy,50.0000


By combining this dataframe, with our earlier results before removing the constraint, we can compare the difference in the volume dispatched. For convenience we will add a column here for the MW change in each unit output. 

For this particular example, **removing the N>>N-NIL_94T_947 increases the dispatched output for only 1 of the 8 units**. Here there is a ~17MW increase in Beryl Solar Farm.
> <font color=purple>Key Takeaway: Not all unit outputs will change (increase) their dispatch equally or proportionally to their coefficients. Remember there are many other factors determining the economic dispatch outcome, some key ones here most likely unit bidding behaviour and other binding constraints which may also apply to these specific units.</font>

In [26]:
dispatch_change = pd.merge(left=units_lhs_vals,right=units_lhs_vals_r,on=['unit','service'])
dispatch_change.rename(columns = {'dispatch_x':'MW before','dispatch_y':'MW after'},inplace=True)
dispatch_change['MW change'] = dispatch_change['MW after'] - dispatch_change['MW before']
dispatch_change

,unit,service,MW before,MW after,MW change
0,BERYLSF1,energy,64.437253,81.1280,16.690747
1,BODWF1,energy,42.525000,42.5250,0.000000
2,GOONSF1,energy,69.000000,69.0000,0.000000
3,JEMALNG1,energy,14.998500,14.9985,0.000000
4,MANSLR1,energy,0.000000,0.0000,0.000000
5,NEVERSF1,energy,98.440000,98.4400,0.000000
6,NYNGAN1,energy,101.043000,101.0430,0.000000
7,PARSF1,energy,50.000000,50.0000,0.000000


---
### C3. Assessing the impact of the constraint removal - Price Outcomes
Even though we only find one unit to be 'relieved' by this constraint, it is not impossible that market prices have changed. We can again merge the two dataframes, one from before and one from after the constraint removal, to quickly verify if alleviating the constraint in question has any material outcome on prices.


In [27]:
prices_r = market.get_energy_prices()
prices_r

,region,price
0,NSW1,34.447395
1,QLD1,32.510394
2,SA1,4.646826
3,TAS1,32.684000
4,VIC1,5.010000


In [28]:
prices_change = pd.merge(left=prices,right=prices_r,on='region')
prices_change.rename(columns = {'price_x':'Price before','price_y':'Price after'},inplace=True)
prices_change['Price change'] = prices_change['Price after'] - prices_change['Price before']
prices_change

,region,Price before,Price after,Price change
0,NSW1,34.616510,34.447395,-1.691151e-01
1,QLD1,32.670000,32.510394,-1.596056e-01
2,SA1,4.646826,4.646826,8.881784e-16
3,TAS1,32.684000,32.684000,0.000000e+00
4,VIC1,5.010000,5.010000,0.000000e+00


Hence, there is no material change to prices in this single dispatch interval example, but there certainly are examples of some constraints having a drastic influence on price outcomes. One potential exercise could be to explore for example stability constraints in this regard.

---
### C4. Assessing the impact of the constraint removal - Elsewhere
A final note is to highlight that specific constraints have a widespread impact on the NEM dispatch and do not only affect those who are involved in the LHS of the constraint.

We will now group the unit dispatch outcomes from before and after the constraint removal for all units that have a noticable change (using a tolerance here of 0.1MW).


In [29]:
diff_all_units = abs(units_r['dispatch']-units['dispatch'])
u_before_change = units[diff_all_units > 1]
u_after_change = units_r[diff_all_units > 1]

u_all_change = pd.merge(left=u_before_change,right=u_after_change,on=['unit','service'])
u_all_change.rename(columns = {'dispatch_x':'MW before','dispatch_y':'MW after'},inplace=True)
u_all_change['MW change'] = u_all_change['MW after'] - u_all_change['MW before']
u_all_change

,unit,service,MW before,MW after,MW change
0,BERYLSF1,energy,64.437253,81.128000,16.690747
1,LOYYB1,raise_reg,6.950248,1.180011,-5.770237
2,LYA1,energy,358.823930,350.000000,-8.823930
3,LYA2,energy,453.000000,449.023196,-3.976804
4,MPP_2,energy,428.178382,422.408144,-5.770237
5,MPP_2,raise_reg,6.571618,12.341856,5.770237
6,SUNRSF1,energy,26.118239,27.149271,1.031032


Interestingly enough, we find two other units have a change in their dispatch.

Binding constraints found in the original dispatch but no longer in the dispatch where N>>N-NIL_94T_947 is removed.

In [30]:
marginal_values[~marginal_values['set'].isin(marginal_values_r['set'])]

,set,constraint_id,type,rhs,slack,marginal_value
24,#MWPS2PV1_E,2420,<=,0.000000,-0.0,-1004.646826
31,#WARWSF1_E,2427,<=,16.000000,0.0,-1032.670000
177,N>>N-NIL_94T_947,2573,<=,131.291525,0.0,-492.298968


Binding constraints found in the new dispatch but were not present in the original dispatch.

In [31]:
marginal_values_r[~marginal_values_r['set'].isin(marginal_values['set'])]

,set,constraint_id,type,rhs,slack,marginal_value
35,$BLOWERNG,2431,=,33.0,0.0,-1034.447395


No direct conclusion can attribute these changes to removing the constraint without further investigation. It would be wise to consider dynamic processes in the dispatch process, i.e. dispatch interval data dependent on previous calculation of economic dispatch. For example the possibility for dynamic RHS values to change or ramp rates etc.

---
## Final Remarks

This example very much highlights the mechanics of constraints as unpacked in a NEM dispatch simulator, nempy, but in essence the key takeaways here are to flag some of the complexities of the dispatch process that cannot be underestimated. One should never assume that by removing a constraint, all solar farms for example would be alleviated. Here we find only 1 of 8 actually increase their volume (MW) of cleared dispatch. Finally, not to mention possible misinterpretation of marginal value and its irrelevance to actual regional price outcomes. In this particular example, no material change in price occured despite the constraint being removed and the 'realisation' of reduced total system cost.

I hope this example proved to be benefiticial in one way or another. Feel free to reach out if you have any questions!